# AgentKit SDK Tutorial 01: 从零开始

本教程面向 **第一次接触 AgentKit SDK 的新手用户**，在一个 Jupyter Notebook 中，带你完成：

1. 安装与环境准备
2. 使用 `sdk.init_project` 初始化一个示例 Agent 项目
3. 使用 `sdk.build` 构建 Agent 镜像
4. 使用 `sdk.deploy` 部署 Agent
5. 使用 `sdk.invoke` 调用已部署的 Agent
6. 使用 `sdk.status` 查看运行状态，并简单清理资源

你可以直接从上到下依次运行每个代码单元。建议在 **仓库根目录** 或已安装好 `agentkit` 的 Python 环境中运行。


## 0. 环境准备

在开始本教程之前，请确保你已经**正确配置了凭证**。AgentKit 需要两类凭证：

### 第一类：SDK 与 CLI 的云服务访问凭证
用于创建和管理云资源（CR、CP、TOS、Runtime 等）。推荐配置方式：

```bash
# 使用 agentkit 命令行工具全局配置（推荐）
agentkit config --global --set volcengine.access_key=YOUR_ACCESS_KEY
agentkit config --global --set volcengine.secret_key=YOUR_SECRET_KEY
agentkit config --global --set volcengine.region=cn-beijing
```

### 第二类：Agent 运行时的环境变量
用于 Agent 访问大模型等服务。在 `agentkit.yaml` 的 `runtime_envs` 中配置(Cloud/Hybrid模式无需配置，Local模式需配置)：

```python
# 模型访问凭证（示例）
config.add_runtime_env("MODEL_AGENT_NAME", "your-model-name")
config.add_runtime_env("MODEL_AGENT_API_KEY", "your-model-api-key")
```

> **注意**：如果你还没有配置凭证，请先学习 `00_configuration_and_credentials.ipynb` 教程完成配置。

此外，请确保：
- 已安装 Python 3.10+
- 已安装 Jupyter / VSCode / PyCharm 等可以运行 Notebook 的环境
- 已安装 AgentKit SDK（通常在同一个代码仓库中开发，或通过 pip 安装）：

```bash
pip install agentkit-sdk-python  # 如果你是独立使用 SDK
```

如果你是在 AgentKit 官方仓库中打开本 Notebook，一般已经满足依赖，只需要确认当前工作目录在仓库根目录即可。


In [ ]:
# 导入本教程会用到的标准库和 AgentKit SDK
from pathlib import Path

from agentkit.toolkit import sdk
from agentkit.toolkit.models import PreflightMode

print("AgentKit SDK 已导入，版本环境就绪。")


## 1. 初始化一个示例 Agent 项目

我们先用 `sdk.init_project` 在当前仓库下创建一个新的示例项目：

- 项目目录：`tutorial_projects/t01-hello-agent`
- 模板：`basic`（最简单的 Python Agent 模板）

`init_project` 会生成：

- `agentkit.yaml` 配置文件
- 示例 Agent 源代码文件
- 一些基础依赖文件（如 `requirements.txt`）等。


In [ ]:
# 约定一个教程专用的项目路径（project_root 为项目根目录）
from pathlib import Path

project_root = Path("tutorial_projects/t01-hello-agent")
project_name = project_root.name

print(f'项目将创建在: {project_root.resolve()}')

# 调用 SDK 的 init_project 创建示例工程（会在 project_root 下生成配置和代码文件）
result = sdk.init_project(
    project_name=project_name,
    template="basic",
    project_root=str(project_root),
)

print("初始化是否成功:", result.success)
print("项目路径:", result.project_path)
print("生成的文件列表:")
for f in result.created_files:
    print(" -", f)

# 使用 init_project 返回的 project_path 作为后续操作的项目根目录
project_root = Path(result.project_path)


# 定位到刚刚生成的配置文件
config_file = project_root / "agentkit.yaml"
print("使用的配置文件:", config_file)

## 2. 构建 Agent 镜像（sdk.build）

项目创建完成后，我们可以使用 `sdk.build` 来构建 Agent 的镜像：

- 通过 `config_file` 参数指定刚才生成的 `agentkit.yaml`
- 使用 `PreflightMode.WARN` 作为预检模式：
  - 如果需要的云服务尚未开通，会给出告警，但仍然继续执行（更适合 SDK 场景）

构建成功后，会返回一个 `BuildResult` 对象，其中包含镜像信息和构建日志。


In [ ]:
# 调用 sdk.build 进行构建
build_result = sdk.build(
    config_file=str(config_file),
    preflight_mode=PreflightMode.WARN,
)

print("构建是否成功:", build_result.success)
print("镜像信息 image:", build_result.image)
print("错误信息 error:", build_result.error)

# 如需查看部分构建日志，可以打印最后若干行
if build_result.build_logs:
    print("构建日志示例 (最后 10 行):")
    for line in build_result.build_logs[-10:]:
        print(line)


## 3. 部署 Agent（sdk.deploy）并调用（sdk.invoke）

构建完成后，可以使用 `sdk.deploy` 将 Agent 部署到：

- 本地 Docker（`launch_type: local`）
- 云端 / 混合模式（根据 `agentkit.yaml` 中的 `launch_type` 配置）

接着使用 `sdk.invoke` 调用已部署的服务。下面的代码假设你的环境已正确配置好对应的运行方式。


In [ ]:
# 部署 Agent
deploy_result = sdk.deploy(
    config_file=str(config_file),
    preflight_mode=PreflightMode.WARN,
)

print("部署是否成功:", deploy_result.success)
print("服务 endpoint_url:", deploy_result.endpoint_url)
print("错误信息 error:", deploy_result.error)


In [ ]:
# 调用已部署的 Agent
from agentkit.utils.logging_config import setup_cli_logging, setup_sdk_logging
setup_sdk_logging("INFO")
invoke_result = sdk.invoke(
    payload={"prompt": "请用一句话向我介绍一下 AgentKit SDK。"},
    headers={"user_id": "user-1", "session_id": "session-1"},
    config_file=str(config_file),
)

print("调用是否成功:", invoke_result.success)
print("响应内容 response:")
print(invoke_result.response)

## 4. 查看运行状态 & 简单清理（status / destroy）

在实际使用中，你可能需要：

- 使用 `sdk.status` 查看当前服务是否在运行
- 使用 `sdk.destroy` 销毁服务及相关资源（不可逆操作）

下面给出一个简单示例，你可以按需选择执行。

In [ ]:
# 查询状态
status_result = sdk.status(config_file=str(config_file))
print("状态查询是否成功:", status_result.success)
print("当前状态 status:", status_result.status)
print("endpoint_url:", status_result.endpoint_url)
print(status_result)



In [ ]:
# 如需完全销毁服务（可选，慎用）
destroy_result = sdk.destroy(config_file=str(config_file))
print("销毁是否成功:", destroy_result.success, destroy_result.error)


## 5. 小结与下一步

在本教程中，你完成了：

1. 使用 `sdk.init_project` 初始化 Agent 项目
2. 使用 `sdk.build` 构建 Agent 镜像，并理解了 `BuildResult` 的基本字段
3. 使用 `sdk.deploy` 部署 Agent，并用 `sdk.invoke` 进行调用
4. 初步了解了 `sdk.status` / `sdk.destroy` 的用途

下一步推荐：

- 阅读 Tutorial 02：使用 `AgentConfig` 和 `AgentKitClient` 管理多次操作
- 结合你自己的业务场景，修改模板生成的 Agent 代码和配置